Source Code for **Automated Caclulation of Social Indicators**

Import libraries

In [32]:
import json
import pandas as pd

Load data structure with WWTP data

In [66]:
with open('data_structure.json','r') as main:
    wwtp_data = json.load(main)

**W2 suitability of wastewater treatment**

Calculate W2 absolute and in percent, unify W2

In [3]:
w2_abs = abs(wwtp_data['plant']['size']['value'] - wwtp_data['influent']['load'])
w2_percent = w2_abs / wwtp_data['plant']['size']['value'] * 100
w2 = 100 - w2_percent

Show value of **W2** [%]

In [1]:
w2

**W3 electricity self-sufficiency**

In [5]:
electricity_self = wwtp_data['energy']['electricity_production_from_sewage_gas']['value'] + \
wwtp_data['energy']['electricity_production_from_PV']['value']

electricity_grid = wwtp_data['energy']['electricity_from_grid']['value'] + electricity_self

w3 = electricity_self / electricity_grid * 100

Show value of **W3** [%]

In [2]:
w3

**E food supply through phosphorus recycling**

Function: Calculate fraction of each sludge disposal path and write to list

In [7]:
def disposal_path_percentage_calc(disposal_dict):

    path_sum = 0
    path_list = []

    for key in disposal_dict.keys():
        if key == 'unit':
            continue
        if key == 'location':
            break

        
        path_sum = path_sum + wwtp_data['sludge disposal'][key]

        path_list.append(wwtp_data['sludge disposal'][key])

      
    percentage_list = []
    for item in range(len(path_list)):
        percentage_list.append(path_list[item] / path_sum)
    
    return(percentage_list)

P recycling: agriculture, humification, mono-incineration<br>
No P recycling: landscaping, co-incineration, landfilling, recultivation

Deploy function

In [28]:
disposal_list = disposal_path_percentage_calc(wwtp_data['sludge disposal'])

Calculate percentage of sewage sludge to P recycling paths (agriculture, humification, mono-incineration)

In [30]:
recycle = (disposal_list[0] + disposal_list[2] + disposal_list[5]) * 100
e = recycle

Show value of **E** [%]

In [3]:
e

**R2 risk of heavy rainfall**

List with considered years (here: 2007 - 2016)

In [3]:
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

**grid data**

Example data [1] from CDC (Germany)

[1] Weather data obtained from: Deutscher Wetterdienst<br>
DWD Climate Data Center (CDC): 
Annual grids of number of days with precipitation >= 30 mm, Version v19.3.<br>
https://cdc.dwd.de/portal/ (accessed 03.11.2020)<br>

For each year:<br>
Click on wanted location in map and select item<br>
Download data as MS Excel [2] and save in folder named "weather_data"<br>
File name must match the name given in path (see below)

In [46]:
days = 0

#loop over 10 years
for year in years:
    #open each file containing wather data of one year
    path = 'weather_data//{} GRD_DEU_P1Y_RR_GE30-.xlsx'.format(year)
    #write to DataFrame
    grid = pd.read_excel(path, 'GRD_DEU_API.GET_FEATURE_INFO_BY')
    
    #sum of number of days with >= 30 mm precipitation in years 2007 - 2016
    days = days + grid.iloc[0, 5]

**station data**

Example data [3] from <br>
[3] CHMI Český hydrometeorologický ústav:<br>
Denní data dle zákona 123/1998 Sb. Denní úhrn srážek<br>
https://www.chmi.cz/historicka-data/pocasi/denni-data/Denni-data-dle-z.-123-1998-Sb# (accessed 25.01.2021)

Download daily precipitation data from weather station nearest to wanted location and save as MS Excel [2] in folder "weather_data"<br>
Write **path** and **sheetname** in respective variables

Write weather data in DataFrame and extract weather data

In [4]:
path = 'weather_data//filename.xlsx'
sheetname = 'sheetname'
station_raw = pd.read_excel(path, sheetname)

end = station_raw.shape[0]

for i in range(40):
    if station_raw.iloc[i, 0] == 'Rok':
        start = i + 1

        station = station_raw.iloc[start:end, 0:4]

Get weather data from ten years (2007 - 2016)

In [5]:
days = 0
for i in range(10):
    for row in range(end):
        if station.iloc[row, 0] < 2007:
            continue
        
        elif station.iloc[row, 0] > 2016:
            break
        
        elif station.iloc[row, 0] == years[i]:
            if station.iloc[row, 3] >= 30:
                    days = days + 1

[2] Microsoft Corporation: Microsoft Excel. 2018

**At this point, the same method is applied regardless of wether grid or station data is available**

Number of days with >= 30 mm precipitation in years 2007 - 2016

In [4]:
days

M: mean value of days with >= 30 mm precipitation in years 2007 - 2016

In [6]:
M = days / 10

Assign R2 according to M

In [23]:
if M < 1 and M >= 0:
    R2 = 0
elif M >= 1 and M < 2:
    R2 = 1
elif M >= 2 and M < 3:
    R2 = 2
elif M >= 3 and M < 4:
    R2 = 3
elif M >= 4:
    R2 = 4

Unify R2 >> percent

In [26]:
if R2 == 0:
    r2 = 100
elif R2 == 1:
    r2 = 75
elif R2 == 2:
    r2 = 50
elif R2 == 3:
    r2 = 25
elif R2 == 4:
    r2 = 0

Show value of **R2** [%]

In [30]:
r2

**C Compliance with wastewater regulation**

This approach is sligthly altered and cosiders only the wastewater regulation [4] of one country (here Germany)

Function: permitted BOD and COD effluent values according to size of WWTP [4]

In [67]:
def get_permitted_value(data_dict):
    
    size = data_dict['plant']['size']['value']
    
    if size < 1000:
        permitted_BOD = 40
        permitted_COD = 150
    
    if size >= 1000 and size <= 5000:
        permitted_BOD = 25
        permitted_COD = 110
        
    if size > 5000 and size <= 10000:
        permitted_BOD = 20
        permitted_COD = 90
    
    if size > 10000 and size <= 100000:
        permitted_BOD = 20
        permitted_COD = 90
    
    
    return permitted_COD, permitted_BOD

[4] Deutscher Bundestag: Abwasserverordnung in Der Fassung Der Bekanntmachung
Vom 17. Juni 2004 (BGBl. I S. 1108, 2625), Die Zuletzt Durch Artikel
1 Der Verordnung Vom 16. Juni 2020 (BGBl. I S. 1287) Geändert Worden
Ist: AbwV. März 1997

Deploy function

In [68]:
permitted_cod, permitted_bod = get_permitted_value(wwtp_data)

Get measured effluent values and compare them to regulation<br>
Give warning if regulation is not met

In [72]:
measured_cod = wwtp_data['effluent']['COD']
measured_bod = wwtp_data['effluent']['BOD']

C2_cod = permitted_cod - measured_cod
C2_bod = permitted_bod - measured_bod

if C2_cod < 0:
    print('Warning: COD value exceeded')
    
if C2_bod < 0:
    print('Warning: BOD value exceeded')

Unify C2 >> percent

In [73]:
c2_cod = C2_cod / permitted_cod * 100
c2_bod = C2_bod / permitted_bod * 100

Show value of **C** [%] for COD and BOD

In [78]:
c2_cod

In [79]:
c2_bod